In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
#%autoreload 2

In [18]:
df = pd.read_csv('p2-arbres-fr.csv', delimiter = ';')
# df

In [24]:
#keeps 5th percentile of values
filtered_df = df[df['genre'].map(df['genre'].value_counts()) > df['genre'].value_counts().quantile(0.8)]
# df.isna().sum()
filtered_df.genre.value_counts()

Platanus             42591
Aesculus             25341
Tilia                21550
Acer                 18471
Sophora              11830
Prunus                7053
Fraxinus              6014
Pinus                 4861
Celtis                4276
Pyrus                 3928
Quercus               3887
Carpinus              3495
Corylus               3463
Populus               3352
Robinia               2317
Betula                2262
Ulmus                 2133
Taxus                 2018
Malus                 1955
Fagus                 1929
Paulownia             1415
Gleditsia             1413
Chamaecyparis         1219
Magnolia              1133
Juglans               1132
Liriodendron          1129
Toona                 1071
Cercis                1019
Pterocarya            1000
Ailanthus              956
Ginkgo                 940
Cedrus                 911
Liquidambar            701
x Cupressocyparis      680
Morus                  679
Name: genre, dtype: int64

In [4]:
df.variete.value_counts()

Baumannii'             4538
Briotii'               2827
Euchlora'              2756
Chanticleer'           2595
Fastigiata'            2483
                       ... 
Conspicua'                1
Concordia'                1
Kojou-no-mai'             1
Nizetii'                  1
Grosse Pomme Noire'       1
Name: variete, Length: 436, dtype: int64

In [26]:
if 'type_emplacement' in df.columns:
    df.drop(columns=['type_emplacement', 'numero'], inplace=True)
    # debatable
    df = df.dropna(subset=['domanialite'])
    df.drop(columns=['complement_addresse'], inplace=True)
    df.remarquable = df.remarquable.apply(lambda x : 1 if x == 1 else 0)
    df.variete = df.variete.fillna('Manquante')
    df['variete'] = df['variete'].apply(lambda x: x[:-1] if x.endswith("'") else x)
    
    ### Filling values for each of genre, species and varietes when there is only one occurence of one of those in
    ### one of those categories.
    
    
    ## repeating the process three times in case one of
    
    
    
    
    # genre > espece
    for g in df.genre.value_counts().index:
        if df[['genre' , 'espece']][df['genre']== g]['espece'].nunique() == 1:
            df.loc[df['genre'] == g, 'espece'] = df.loc[df['genre'] == g, 'espece'].iloc[0]

    # espece > variete
    # espece > variete
    
    
    

In [12]:
df.nunique()

id                     200137
type_emplacement            1
domanialite                 9
arrondissement             25
complement_addresse      3795
numero                      0
lieu                     6921
id_emplacement          69040
libelle_francais          192
genre                     175
espece                    539
variete                   436
circonference_cm          531
hauteur_m                 143
stade_developpement         4
remarquable                 2
geo_point_2d_a         200107
geo_point_2d_b         200114
dtype: int64

In [97]:
df.domanialite.value_counts()

Alignement      104949
Jardin           46262
CIMETIERE        31926
DASCO             6422
PERIPHERIQUE      5327
DJS               3900
DFPE              1325
DAC                 21
DASES                4
Name: domanialite, dtype: int64

In [120]:
df.isna().sum()[df.isna().sum()!=0]

libelle_francais        1497
genre                     16
espece                  1752
stade_developpement    67205
dtype: int64

In [ ]:
df.remarquable.value_counts()

In [ ]:
df.shape[0] == df.espece.value_counts().sum()


In [9]:
df['espece'][df.variete == "Atropunicea'"].value_counts()

sylvatica    102
Name: espece, dtype: int64

In [28]:
df[df.espece=='hippocastanum'].stade_developpement.value_counts()

A     9864
JA    2888
J     1709
M     1367
Name: stade_developpement, dtype: int64

In [29]:
df.isna().sum()

id                         0
domanialite                0
arrondissement             0
lieu                       0
id_emplacement             0
libelle_francais        1497
genre                     16
espece                  1751
variete                    0
circonference_cm           0
hauteur_m                  0
stade_developpement    67205
remarquable                0
geo_point_2d_a             0
geo_point_2d_b             0
dtype: int64

In [117]:
df[df.variete != "Manquante"]

,id,domanialite,arrondissement,lieu,id_emplacement,libelle_francais,genre,espece,variete,circonference_cm,hauteur_m,stade_developpement,remarquable,geo_point_2d_a,geo_point_2d_b
11,99885,Jardin,PARIS 16E ARRDT,JARDIN DU RANELAGH,20001,Hêtre,Fagus,sylvatica,Atropunicea,30,0,NaN,0,48.858222,2.269287
25,99900,Jardin,PARIS 16E ARRDT,JARDIN DU RANELAGH,20021,Hêtre,Fagus,sylvatica,Atropunicea,100,0,NaN,0,48.857849,2.267720
26,99901,Jardin,PARIS 16E ARRDT,JARDIN DU RANELAGH,20022,Hêtre,Fagus,sylvatica,Atropunicea,35,0,NaN,0,48.857803,2.267476
27,99902,Jardin,PARIS 16E ARRDT,JARDIN DU RANELAGH,20023,Hêtre,Fagus,sylvatica,Atropunicea,185,0,NaN,0,48.857809,2.267599
28,99903,Jardin,PARIS 16E ARRDT,JARDIN DU RANELAGH,20024,Hêtre,Fagus,sylvatica,Atropunicea,130,0,NaN,0,48.857790,2.267713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198835,2023213,DJS,PARIS 19E ARRDT,CENTRE SPORTIF JULES LADOUMEGUE / 35 ROUTE DES...,109002,Saule,Salix,babylonica,Tortuosa,20,5,NaN,0,48.890661,2.397392
198866,2023360,Alignement,PARIS 1ER ARRDT,RUE DE TURBIGO,402001,Fevier,Gleditsia,triacanthos,Skyline,20,5,NaN,0,48.863689,2.348325
198867,2023361,Alignement,PARIS 1ER ARRDT,RUE DE TURBIGO,402002,Fevier,Gleditsia,triacanthos,Skyline,20,5,NaN,0,48.863716,2.348449
198868,2023362,Alignement,PARIS 1ER ARRDT,RUE DE TURBIGO,402003,Fevier,Gleditsia,triacanthos,Skyline,20,5,NaN,0,48.863744,2.348573


In [11]:
np.round(df.genre.value_counts().sum())/10

20012.1

In [12]:
df.genre.value_counts().mean()

1143.5485714285714

In [14]:
# <ods-dataset-context context="lesarbresplantes" lesarbresplantes-dataset="les-arbres-plantes" lesarbresplantes-parameters="{'geofilter.polygon':'(48.88639177703194,2.228851318359375),(48.812289858662574,2.230224609375),(48.7874140472094,2.349700927734375),(48.82268881260476,2.4863433837890625),(48.94054322456003,2.4217987060546875),(48.88639177703194,2.228851318359375)'}">
#     <ods-map no-refit="true" scroll-wheel-zoom="false" display-control="true" search-box="true" toolbar-fullscreen="true" toolbar-geolocation="true" basemap="jawg.streets" location="13,48.89886,2.34249">
#         <ods-map-layer-group>
#             <ods-map-layer context="lesarbresplantes" color="#C32D1C" picto="ods-circle" show-marker="true" display="auto" shape-opacity="0.5" point-opacity="1" border-color="#FFFFFF" border-opacity="1" border-size="1" border-pattern="solid" caption="true" title="Les arbres plantés" description="Jeu de données géographique
#     présentant les arbres plantés (hors peuplement forestiers)" size="4" size-min="3" size-max="5" size-function="linear"></ods-map-layer>
#         </ods-map-layer-group>
#     </ods-map>

# </ods-dataset-context>

In [15]:
df['variete'].dropna().apply(lambda x: isinstance(x, float)).value_counts()


False    36777
Name: variete, dtype: int64

In [16]:
df.genre.value_counts()[df.genre.value_counts()<3].count()

39

In [17]:
df[['genre' , 'espece']][df['genre']=='Taxus'].value_counts()

genre  espece          
Taxus  baccata             1991
       n. sp.                 7
       alba                   4
       x media                2
       baccata f. Aurea       1
dtype: int64

In [18]:
df.espece.value_counts()

x hispanica                  36409
hippocastanum                20039
japonica                     11822
n. sp.                        9063
tomentosa                     8962
                             ...  
polycarpa                        1
x gondouinii                     1
lusitanica subsp.azorica         1
oliveri                          1
delavayi subsp. potaninii        1
Name: espece, Length: 539, dtype: int64

In [19]:
df[['genre' , 'espece']][df['genre']=='Taxus']['espece'].value_counts()

baccata             1991
n. sp.                 7
alba                   4
x media                2
baccata f. Aurea       1
Name: espece, dtype: int64

In [20]:
df.espece.value_counts().index

Index(['x hispanica', 'hippocastanum', 'japonica', 'n. sp.', 'tomentosa',
       'pseudoplatanus', 'platanoides', 'nigra', 'x europaea', 'x carnea',
       ...
       'x chinensis', 'minor var. vulgaris', 'x macrantha', 'acerifolius',
       'horizontalis', 'polycarpa', 'x gondouinii', 'lusitanica subsp.azorica',
       'oliveri', 'delavayi subsp. potaninii'],
      dtype='object', length=539)

In [42]:
df[df.genre == 'Liriodendron'].genre.value_counts()

Liriodendron    1129
Name: genre, dtype: int64

In [36]:
g

'Liriodendron'

In [ ]:
df.espece = df.espece.apply(lambda x : df.genre[df.genre == g].value_counts().index[0])

2. ***Pistes à suivre***

In [48]:
# df.id_emplacement.value_counts()
# Creuser 'Manquante'


* DUMP

In [80]:
for g in df.genre.value_counts().index:
    if df[['genre' , 'espece']][df['genre']== g]['espece'].nunique() == 1:
        df.loc[df['genre'] == g, 'espece'] = df.loc[df['genre'] == g, 'espece'].iloc[0]
#         df.espece = df.espece.apply(lambda x : df.genre[df.genre == g].value_counts()[0])


# fonctionne mais ne retire qu'une espece